In [1]:
import pandas as pd

In [2]:
df = pd.read_json("../data/LRE_corpus/LRE_corpus-0.1/corpora.json", lines=True)

In [3]:
df

,id_str,lre_annotation
0,527760967652749312,[]
1,527007061029056512,[]
2,528154012315426816,[]
3,528220003691024384,[]
4,526282301366226944,[]
...,...,...
9995,507167181210017792,[]
9996,529628482158878720,[]
9997,528209131945984000,[]
9998,521454722234851328,[]


In [4]:
# Proportion of datapoints where lre_annotation column is an empty list
df.lre_annotation.apply(lambda x: len(x) != 0).mean() * len(df)

793.0

In [5]:
from pandas import json_normalize

In [6]:
df['lre_annotation'][38]

[{'start': 6,
  'end': 11,
  'surface': 'MOVIX',
  'note': '',
  'entityClass': 'facility',
  'entities': [{'properties': {'url': 'http://loco.yahoo.co.jp/place/649c171a7fdb49261e4700d1501b93b8c022f067/'},
    'geometry': {'coordinates': [None, None], 'type': 'Point'},
    'type': 'Feature'}]},
 {'start': 24,
  'end': 26,
  'surface': '利府',
  'note': '',
  'entityClass': 'location',
  'entities': [{'properties': {'altnames': ['利府', '利府町'],
     'note': '',
     'address': '宮城郡利府町利府字新並松４',
     'prefix': '',
     'suffix': '町/',
     'ne_class': '市区町村/町',
     'address_level': 3,
     'code': 'lasdec:044067/jisx0402:04406',
     'entry_id': 2310,
     'hypernym': '宮城県/宮城郡',
     'body': '利府',
     'surface': '利府町'},
    'geometry': {'coordinates': [140.97684, 38.33084], 'type': 'Point'},
    'type': 'Feature'}]},
 {'start': 34,
  'end': 39,
  'surface': 'MOVIX',
  'note': '',
  'entityClass': 'facility',
  'entities': [{'properties': {'url': 'http://loco.yahoo.co.jp/place/649c171a7fdb49

In [32]:
a = json_normalize(df['lre_annotation'][38])
a

,start,end,surface,note,entityClass,entities
0,6,11,MOVIX,,facility,[{'properties': {'url': 'http://loco.yahoo.co....
1,24,26,利府,,location,"[{'properties': {'altnames': ['利府', '利府町'], 'n..."
2,34,39,MOVIX,,facility,[{'properties': {'url': 'http://loco.yahoo.co....


In [33]:
a["entities"] = a["entities"].apply(lambda x: x[0]["geometry"]["coordinates"] if x else None)
a["entities"] = a["entities"].apply(lambda x: tuple(x) if x[0] else None)
a = a.rename(columns={"entities": "coordinates"})

In [34]:
a

,start,end,surface,note,entityClass,coordinates
0,6,11,MOVIX,,facility,None
1,24,26,利府,,location,"(140.97684, 38.33084)"
2,34,39,MOVIX,,facility,None


KeyError: 'entities'

In [22]:
import pandas as pd

# Sample DataFrame
df = pd.DataFrame({'A': ["a", "b", "c"],
                   'B': [1, 2, 3]})

# Custom function that returns a DataFrame
def custom_function(value):
    return pd.DataFrame({'C': [value * 2, value * 3],
                         'D': [value * 4, value * 5]})

# Apply the function to column 'A' using the apply method
result = df['B'].apply(custom_function)

a_df = df['A'].apply(lambda a: [a]*2).explode("A")

# Concatenate the resulting DataFrames
result = pd.concat([result[i] for i in range(len(result))], axis=0).reset_index(drop=True)
result = pd.concat([a_df,result], axis=1)

print(result)

   A  C   D
0  a  2   4
1  a  3   5
2  b  4   8
3  b  6  10
4  c  6  12
5  c  9  15


In [6]:
import pandas as pd
from pandas import json_normalize

df = pd.read_json("../data/LRE_corpus/LRE_corpus-0.1/corpora.json", lines=True)

In [7]:
df

,id_str,lre_annotation
0,527760967652749312,[]
1,527007061029056512,[]
2,528154012315426816,[]
3,528220003691024384,[]
4,526282301366226944,[]
...,...,...
9995,507167181210017792,[]
9996,529628482158878720,[]
9997,528209131945984000,[]
9998,521454722234851328,[]


In [8]:


def custom_function(value):
    if not value:
        return None
    # print(value)
    a = json_normalize(value)
    # print(a)
    a["entities"] = a["entities"].apply(lambda x: x[0]["geometry"]["coordinates"] if x else None)
    a["entities"] = a["entities"].apply(lambda x: tuple(x) if x and x[0] else None)
    a = a.rename(columns={"entities": "coordinates"})
    return a

result = df['lre_annotation'].apply(custom_function)


In [9]:

# Concatenate the resulting DataFrames
result = pd.concat([result[i] for i in range(len(result))], axis=0).reset_index(drop=True)

result

,start,end,surface,note,entityClass,coordinates
0,19,24,ファミレス,,generic,None
1,6,11,MOVIX,,facility,None
2,24,26,利府,,location,"(140.97684, 38.33084)"
3,34,39,MOVIX,,facility,None
4,19,21,お店,複数の特定不能店舗,facility-UNSP,None
...,...,...,...,...,...,...
1039,22,23,泉,,location-UNSP,None
1040,0,3,アキバ,,facility,None
1041,7,12,ラーメン屋,,facility-UNSP,None
1042,33,36,森ビル,,facility,None


In [11]:
# proportion of results with non-null coordinates
result.coordinates.apply(lambda x: x is not None).mean()*len(result)

299.0